$$|\psi\rangle=\cos\left(\frac{\theta}{2}\right)|0\rangle+\sin\left(\frac{\theta}{2}\right)e^{i\phi}|1\rangle.$$

In [1]:
import numpy as np

# 量子态的表示
def quantum_state(theta, phi):

    coeff_0 = np.cos(theta / 2)
    coeff_1 = np.sin(theta / 2) * np.exp(1j * phi)
    return coeff_0 * np.array([1, 0]) + coeff_1 * np.array([0, 1])

# import random
# # 设定参数
# theta = random.uniform(0, np.pi)    # θ的值
# phi = random.uniform(0, 2 * np.pi)  # φ的值

# # 计算量子态
# psi = quantum_state(theta, phi)

# print("|ψ⟩ =", psi)


$$\hat{\rho}=\sum_i P_i|\psi_i\rangle\langle\psi_i|,~\sum_i P_i =1$$

In [2]:
import numpy as np
import random

# generate density martrix randomly
def generate_density_matrix(num_qubit=2, if_pure_state_included=False):
    
    if if_pure_state_included:
        num_state = random.randint(1, 3)**num_qubit
    else:
        num_state = random.randint(2, 3)**num_qubit 
    possible_each_states = np.random.rand(num_state)
    possible_each_states /= sum(possible_each_states)

    if num_state == 1:
        if_pure = 1
    else:
        if_pure = 0

    for i in range(num_state):
        state = 1
        for j in range(num_qubit):
            theta = random.uniform(0, np.pi) 
            phi = random.uniform(0, 2 * np.pi)
            state_single = quantum_state(theta, phi)
            state = np.kron(state_single, state) 
        if i == 0:
            density_matrix = np.outer(state, state.conj()) * possible_each_states[i]
        else:
            density_matrix += np.outer(state, state.conj()) * possible_each_states[i]
    
    return density_matrix, possible_each_states, if_pure

$$\rho_A=Tr_B(\rho_{AB})$$

$$\mathrm{Tr}_B(C)=\sum_{i=0}^{n-1}{(I_n\otimes\langle i|)C(I_n\otimes|i\rangle)}$$

$$\mathrm{Tr}_A(C)=\sum_{i=0}^{n-1}\left(\langle i|\otimes I_n\right)C(|i\rangle\otimes I_n)$$



In [3]:
# 求偏迹
def partial_trace_A(rho_AB, dim_A, dim_B):

    rho_A = np.zeros((dim_A, dim_A), dtype=complex)
    for i in range(dim_B):
        # |i>
        ket_i = np.zeros((dim_B,1), dtype=complex)
        ket_i[i] = 1
        I_A = np.eye(dim_A)

        rho_A += np.kron(I_A, ket_i.conj().T) @ rho_AB @ np.kron(I_A, ket_i)
    
    return rho_A

def partial_trace_B(rho_AB, dim_A, dim_B):

    rho_B = np.zeros((dim_B, dim_B), dtype=complex)
    for i in range(dim_A):
        # |i>
        ket_i = np.zeros((dim_B,1), dtype=complex)
        ket_i[i] = 1
        I_B = np.eye(dim_B)

        rho_B += np.kron(ket_i.conj().T,I_B) @ rho_AB @ np.kron(ket_i, I_B)
    
    return rho_B

In [4]:
rho_AB, p_AB, _ = generate_density_matrix(num_qubit=4, if_pure_state_included=True)
rho_AB, p_AB

(array([[ 2.09198484e-03+0.00000000e+00j,  1.31652471e-03-2.13688683e-03j,
          9.38295775e-04+1.06618190e-02j,  1.14811493e-02+5.75124449e-03j,
          5.06128778e-04-1.99870759e-03j, -1.72309130e-03-1.77481584e-03j,
          1.04134393e-02+1.68303063e-03j,  8.27252459e-03-9.57778921e-03j,
          2.89862650e-03-1.79701250e-03j, -1.14240510e-05-4.09173526e-03j,
          1.04585801e-02+1.39668803e-02j,  2.08484406e-02-1.89344540e-03j,
         -1.01560211e-03-3.20414679e-03j, -3.91205719e-03-9.79027948e-04j,
          1.58744449e-02-6.61314703e-03j,  3.23499098e-03-2.03769467e-02j],
        [ 1.31652471e-03+2.13688683e-03j,  3.01126590e-03+6.92558735e-20j,
         -1.03001756e-02+7.66811487e-03j,  1.35061116e-03+1.53469430e-02j,
          2.36012368e-03-7.40831371e-04j,  7.28536986e-04-2.87699982e-03j,
          4.83421481e-03+1.16961137e-02j,  1.49894177e-02+2.42260490e-03j,
          3.65974245e-03+1.82994894e-03j,  4.17237016e-03-2.58667384e-03j,
         -7.68488529e-03

In [5]:
np.trace(rho_AB)

(1+2.109866420183247e-18j)

In [6]:
eigenvalues, _ = np.linalg.eig(rho_AB)
eigenvalues

array([-4.33227005e-19-5.67638780e-20j,  1.00000000e+00+6.99565778e-18j,
        1.64276594e-17+9.71500232e-18j,  6.45235761e-19-1.67608593e-17j,
       -1.70287243e-17+6.58839188e-18j,  1.38699471e-17-1.35128867e-18j,
       -1.16362780e-17+1.79144630e-18j, -9.13637045e-18+5.96989739e-18j,
       -7.57742999e-18-9.45858619e-19j,  2.88319563e-18-3.46456726e-18j,
       -2.71484808e-18+5.31561291e-19j,  2.13993660e-18+5.42368276e-19j,
        2.34770715e-19-8.78229125e-19j, -4.16450094e-19+9.61146282e-19j,
       -3.26377398e-19-6.41068546e-20j,  4.63565738e-19+4.55717031e-19j])

In [7]:
rho_A = partial_trace_A(rho_AB, 4, 4)
rho_A

array([[ 0.13868375+5.74994499e-19j,  0.03355275-1.32500133e-01j,
         0.19215837-1.19129179e-01j, -0.06732721-2.12412200e-01j],
       [ 0.03355275+1.32500133e-01j,  0.13470988+2.90836811e-18j,
         0.1603077 +1.54768660e-01j,  0.18665223-1.15715629e-01j],
       [ 0.19215837+1.19129179e-01j,  0.1603077 -1.54768660e-01j,
         0.36858392+2.05499754e-18j,  0.08917413-3.52149529e-01j],
       [-0.06732721+2.12412200e-01j,  0.18665223+1.15715629e-01j,
         0.08917413+3.52149529e-01j,  0.35802245-3.42849373e-18j]])

In [8]:
np.trace(rho_A)

(1+2.109866420183247e-18j)

In [9]:
rho_B = partial_trace_B(rho_AB, 4, 4)
rho_B

array([[ 0.01508457-3.05802822e-20j,  0.009493  -1.54083429e-02j,
         0.00676572+7.68786451e-02j,  0.08278655+4.14702109e-02j],
       [ 0.009493  +1.54083429e-02j,  0.02171318+2.18705000e-19j,
        -0.07427096+5.52920922e-02j,  0.00973878+1.10661434e-01j],
       [ 0.00676572-7.68786451e-02j, -0.07427096-5.52920922e-02j,
         0.39484725+5.47359423e-19j,  0.24848467-4.03322175e-01j],
       [ 0.08278655-4.14702109e-02j,  0.00973878-1.10661434e-01j,
         0.24848467+4.03322175e-01j,  0.568355  +1.37438228e-18j]])

In [10]:
np.trace(rho_B)

(1+2.109866420183247e-18j)

In [11]:
np.kron(rho_A, rho_B)

array([[ 2.09198484e-03+4.43255672e-21j,  1.31652471e-03-2.13688683e-03j,
         9.38295775e-04+1.06618190e-02j,  1.14811493e-02+5.75124449e-03j,
         5.06128778e-04-1.99870759e-03j, -1.72309130e-03-1.77481584e-03j,
         1.04134393e-02+1.68303063e-03j,  8.27252459e-03-9.57778921e-03j,
         2.89862650e-03-1.79701250e-03j, -1.14240510e-05-4.09173526e-03j,
         1.04585801e-02+1.39668803e-02j,  2.08484406e-02-1.89344540e-03j,
        -1.01560211e-03-3.20414679e-03j, -3.91205719e-03-9.79027948e-04j,
         1.58744449e-02-6.61314703e-03j,  3.23499098e-03-2.03769467e-02j],
       [ 1.31652471e-03+2.13688683e-03j,  3.01126590e-03+4.28157919e-20j,
        -1.03001756e-02+7.66811487e-03j,  1.35061116e-03+1.53469430e-02j,
         2.36012368e-03-7.40831371e-04j,  7.28536986e-04-2.87699982e-03j,
         4.83421481e-03+1.16961137e-02j,  1.49894177e-02+2.42260490e-03j,
         3.65974245e-03+1.82994894e-03j,  4.17237016e-03-2.58667384e-03j,
        -7.68488529e-03+1.94726770e-0

In [12]:
rho_AB

array([[ 2.09198484e-03+0.00000000e+00j,  1.31652471e-03-2.13688683e-03j,
         9.38295775e-04+1.06618190e-02j,  1.14811493e-02+5.75124449e-03j,
         5.06128778e-04-1.99870759e-03j, -1.72309130e-03-1.77481584e-03j,
         1.04134393e-02+1.68303063e-03j,  8.27252459e-03-9.57778921e-03j,
         2.89862650e-03-1.79701250e-03j, -1.14240510e-05-4.09173526e-03j,
         1.04585801e-02+1.39668803e-02j,  2.08484406e-02-1.89344540e-03j,
        -1.01560211e-03-3.20414679e-03j, -3.91205719e-03-9.79027948e-04j,
         1.58744449e-02-6.61314703e-03j,  3.23499098e-03-2.03769467e-02j],
       [ 1.31652471e-03+2.13688683e-03j,  3.01126590e-03+6.92558735e-20j,
        -1.03001756e-02+7.66811487e-03j,  1.35061116e-03+1.53469430e-02j,
         2.36012368e-03-7.40831371e-04j,  7.28536986e-04-2.87699982e-03j,
         4.83421481e-03+1.16961137e-02j,  1.49894177e-02+2.42260490e-03j,
         3.65974245e-03+1.82994894e-03j,  4.17237016e-03-2.58667384e-03j,
        -7.68488529e-03+1.94726770e-0

In [13]:
# 如果没有文件夹，创建文件夹
import os

data_dir = 'data_mixed_16+'
dir = [data_dir + '/rho_AB', data_dir + '/rho_A', data_dir + '/rho_B', 
       data_dir + '/entropy_A', data_dir + '/entropy_B', data_dir + '/entropy_AB',
       data_dir + '/I2', data_dir + '/all', data_dir + '/if_pure',
       data_dir + '/observable0', data_dir + '/observable1', 
       data_dir + '/observable2', data_dir + '/observable3',
       data_dir + '/observable4', data_dir + '/observable5',
       data_dir + '/observable6', data_dir + '/observable7',
       data_dir + '/observable8', data_dir + '/observable9',
       data_dir + '/observable10', data_dir + '/observable11',
       data_dir + '/observable12', data_dir + '/observable13',
       data_dir + '/observable14', data_dir + '/observable15']
for i in dir:
    if not os.path.exists(i):
        os.makedirs(i)


$$I_2(A:B)=S\left(\rho_A\right)+S\left(\rho_B\right)-S\left(\rho_{AB}\right),$$
$$S\left(\rho\right)=-\sum_i\left(\lambda_i\log\lambda_i\right)$$
$$o_i(\rho)=\mathrm{trace}(O_i\rho)$$

In [14]:
from tqdm import tqdm

num_sample = 100000
for i in tqdm(range(num_sample)):
    rho_AB, p_AB, if_pure = generate_density_matrix(num_qubit=2, if_pure_state_included=False)
    rho_A = partial_trace_A(rho_AB, 2, 2)
    rho_B = partial_trace_B(rho_AB, 2, 2)

    eigenvalue_AB, _ = np.linalg.eig(rho_AB)
    eigenvalue_A, _ = np.linalg.eig(rho_A)
    eigenvalue_B, _ = np.linalg.eig(rho_B)

    entropy_A = -np.sum(eigenvalue_A * np.log(eigenvalue_A)).real
    entropy_B = -np.sum(eigenvalue_B * np.log(eigenvalue_B)).real
    entropy_AB = -np.sum(eigenvalue_AB * np.log(eigenvalue_AB)).real
    # 让NAN、INF等变为0
    entropy_A = np.nan_to_num(entropy_A)
    entropy_B = np.nan_to_num(entropy_B)
    entropy_AB = np.nan_to_num(entropy_AB)
    
    I2 = entropy_A + entropy_B - entropy_AB

    i0 = np.array([[1, 0], [0, 1]]) # I
    a0 = np.array([[0, 1], [1, 0]]) # sigma_x
    a1 = np.array([[1, 0], [0, -1]]) # sigma_z
    a2 = np.array([[0, -1j], [1j, 0]]) # sigma_y


    observable0 = np.trace( np.kron(i0, i0) @ rho_AB ).real
    observable1 = np.trace( np.kron(a0, i0) @ rho_AB ).real
    observable2 = np.trace( np.kron(a1, i0) @ rho_AB ).real
    observable3 = np.trace( np.kron(a2, i0) @ rho_AB ).real
    observable4 = np.trace( np.kron(i0, a0) @ rho_AB ).real
    observable5 = np.trace( np.kron(i0, a1) @ rho_AB ).real
    observable6 = np.trace( np.kron(i0, a2) @ rho_AB ).real
    observable7 = np.trace( np.kron(a0, a0) @ rho_AB ).real
    observable8 = np.trace( np.kron(a1, a1) @ rho_AB ).real
    observable9 = np.trace( np.kron(a2, a2) @ rho_AB ).real
    observable10 = np.trace( np.kron(a0, a1) @ rho_AB ).real
    observable11 = np.trace( np.kron(a0, a2) @ rho_AB ).real
    observable12 = np.trace( np.kron(a1, a0) @ rho_AB ).real
    observable13 = np.trace( np.kron(a1, a2) @ rho_AB ).real
    observable14 = np.trace( np.kron(a2, a0) @ rho_AB ).real
    observable15 = np.trace( np.kron(a2, a1) @ rho_AB ).real


    np.savetxt(data_dir + '/rho_AB/'+ "{:08d}".format(i)+'.txt', rho_AB)
    np.savetxt(data_dir + '/rho_A/'+ "{:08d}".format(i)+'.txt', rho_A)
    np.savetxt(data_dir + '/rho_B/'+ "{:08d}".format(i)+'.txt', rho_B)
    np.savetxt(data_dir + '/entropy_AB/'+ "{:08d}".format(i)+'.txt', [entropy_AB])
    np.savetxt(data_dir + '/entropy_A/'+ "{:08d}".format(i)+'.txt', [entropy_A])
    np.savetxt(data_dir + '/entropy_B/'+ "{:08d}".format(i)+'.txt', [entropy_B])
    np.savetxt(data_dir + '/I2/'+ "{:08d}".format(i)+'.txt', [I2])
    np.savetxt(data_dir + '/if_pure/'+ "{:08d}".format(i)+'.txt', [if_pure])
    np.savetxt(data_dir + '/observable0/'+ "{:08d}".format(i)+'.txt', [observable0])
    np.savetxt(data_dir + '/observable1/'+ "{:08d}".format(i)+'.txt', [observable1])
    np.savetxt(data_dir + '/observable2/'+ "{:08d}".format(i)+'.txt', [observable2])
    np.savetxt(data_dir + '/observable3/'+ "{:08d}".format(i)+'.txt', [observable3])
    
    np.savetxt(data_dir + '/observable4/'+ "{:08d}".format(i)+'.txt', [observable4])
    np.savetxt(data_dir + '/observable5/'+ "{:08d}".format(i)+'.txt', [observable5])
    np.savetxt(data_dir + '/observable6/'+ "{:08d}".format(i)+'.txt', [observable6])
    np.savetxt(data_dir + '/observable7/'+ "{:08d}".format(i)+'.txt', [observable7])
    np.savetxt(data_dir + '/observable8/'+ "{:08d}".format(i)+'.txt', [observable8])
    np.savetxt(data_dir + '/observable9/'+ "{:08d}".format(i)+'.txt', [observable9])
    np.savetxt(data_dir + '/observable10/'+ "{:08d}".format(i)+'.txt', [observable10])
    np.savetxt(data_dir + '/observable11/'+ "{:08d}".format(i)+'.txt', [observable11])
    np.savetxt(data_dir + '/observable12/'+ "{:08d}".format(i)+'.txt', [observable12])
    np.savetxt(data_dir + '/observable13/'+ "{:08d}".format(i)+'.txt', [observable13])
    np.savetxt(data_dir + '/observable14/'+ "{:08d}".format(i)+'.txt', [observable14])
    np.savetxt(data_dir + '/observable15/'+ "{:08d}".format(i)+'.txt', [observable15])

    # np.savetxt(data_dir + '/all/'+ "{:08d}".format(i)+'.txt', 
    #            [observable0, observable1, observable2, observable3, if_pure, I2])
    np.savetxt(data_dir + '/all/'+ "{:08d}".format(i)+'.txt',
                [observable0, observable1, observable2, observable3, observable4, observable5, observable6, observable7, observable8, observable9, observable10, observable11, observable12, observable13, observable14, observable15, if_pure, I2])
    

  1%|          | 6692/1000000 [03:11<7:53:36, 34.96it/s] 


KeyboardInterrupt: 